In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def fetch_all_indicators(credentials_path = "connection-123-892e002c2def.json") -> pd.DataFrame:
        """Fetch and combine all technical indicators from BigQuery"""
        credentials = service_account.Credentials.from_service_account_file(credentials_path)
        client = bigquery.Client(credentials=credentials, project="connection-123")

        # Main query joining all indicator tables
        query = """
        WITH base_data AS (
            SELECT
                p.timestamp as date_,
                p.price,
                p.market_cap,
                p.total_volume,
                ma.sma_10,
                ma.sma_20,
                ma.sma_50,
                ema.ema_9,
                ema.ema_12,
                ema.ema_26,
                ema.ema_20,
                ema.ema_50
            FROM `connection-123.signals.bitcoin_price` p
            LEFT JOIN `connection-123.signals.btc_moving_averages` ma ON p.timestamp = ma.date_
            LEFT JOIN `connection-123.signals.btc_ema` ema ON p.timestamp = ema.timestamp
            LEFT JOIN `connection-123.signals.btc_rsi` rsi ON p.timestamp = rsi.timestamp
            LEFT JOIN `connection-123.signals.btc_macd` macd ON p.timestamp = macd.timestamp
            LEFT JOIN `connection-123.signals.btc_bollinger_bands` bb ON p.timestamp = bb.timestamp
            WHERE p.timestamp BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 180 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
            ORDER BY p.timestamp
        )
        SELECT *
        FROM base_data
        WHERE sma_10 IS NOT NULL  -- Ensure we have indicator data
        """

        query_job = client.query(query)
        df = query_job.result().to_dataframe()

        return df

In [3]:
data = fetch_all_indicators()

In [5]:
X = data.drop(['price','date_'], axis=1)
y = data['price']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [8]:
model = sm.OLS(Y_train, X_train)
results = model.fit()

In [11]:
def predict_next_days(results, days=5):
    """Predict Bitcoin price for next few days using the trained model"""

    # Get the most recent indicator values from your data
    latest_indicators = data.iloc[-1][['market_cap', 'total_volume', 'sma_10', 'sma_20', 'sma_50',
                                      'ema_9', 'ema_12', 'ema_26', 'ema_20', 'ema_50']].values

    predictions = []
    current_indicators = latest_indicators.copy()

    for day in range(1, days + 1):
        # Add constant for prediction
        X_pred = np.insert(current_indicators, 0, 1)  # Add constant at beginning

        # Make prediction
        predicted_price = results.predict(X_pred)[0]
        predictions.append({
            'day': day,
            'predicted_price': predicted_price,
            'date': pd.Timestamp.now() + pd.Timedelta(days=day)
        })

        # Simple assumption: indicators change gradually (you can improve this)
        # For now, assume moving averages slowly converge toward predicted price
        current_indicators[2:5] = current_indicators[2:5] * 0.95 + predicted_price * 0.05  # SMAs
        current_indicators[5:] = current_indicators[5:] * 0.9 + predicted_price * 0.1     # EMAs

    return pd.DataFrame(predictions)

# Generate predictions
future_predictions = predict_next_days(results, days=5)
print("Next 5 Days Bitcoin Price Predictions:")
print(future_predictions[['date', 'predicted_price']].round(2))

Next 5 Days Bitcoin Price Predictions:
                        date  predicted_price
0 2025-09-11 16:44:26.098107        112248.17
1 2025-09-12 16:44:26.099939        112238.37
2 2025-09-13 16:44:26.100053        112229.84
3 2025-09-14 16:44:26.100148        112222.45
4 2025-09-15 16:44:26.100260        112216.06
